In [13]:
import numpy as np
import math
from PIL import Image
from PIL import ImageOps
from PIL import ImageFilter, ImageEnhance
import cv2
import os
from Adaptive_Gamma_Correction import adaptiveGammaCorrection
from Adaptive_CLAHE import adaptiveCLAHE
import matplotlib.pyplot as plt


In [14]:
def wrapper(path):
	""" Extensive Image-Enhancement featuring Adaptive-CLAHE and Adaptive Gamma-Correction. 
  Colors are additonally improved via a simple gradient guided saturation adjustment.
  Can be used as general script for large amount of images e.g. from vacation, parties, shootings...
  
  Due to the implementation, some parts are unperformant python code. Therefore enhancing a huge amount of high-res images
  will take some time. Better grab a coffee ;)

    Args:
    path: Path to image
    Returns:
      Returns extensively enhanced Image.
    """

	#filename = os.path.basename(path)
	#uncomment for duplicating the orignal images 
	#img = Image.open(path).convert('RGB')
	#img.save(saveDirectory + filename + '.png')
	
	img_orig = cv2.imread(path, 1)
	img = img_orig

	hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
	h, s, v = cv2.split(hsv)

	gradient =  cv2.Laplacian(s,cv2.CV_32F, ksize = 1)  #cv2.Laplacian(s, cv2.CV_32F, ksize = 1) 
	clipped_gradient = gradient * np.exp(-1 * np.abs(gradient) * np.abs(s - 0.5))

	#normalize to [-1...1]
	clipped_gradient =  2*(clipped_gradient - np.max(clipped_gradient))/-np.ptp(clipped_gradient)-1
	#clipped_gradient = (clipped_gradient - np.amin(clipped_gradient)) / (np.amax(clipped_gradient) - np.amin(clipped_gradient)) 
	clipped_gradient =  0.5 * clipped_gradient #--> 0.5 limits maximum saturation change to 50 %

	factor = np.add(1.0, clipped_gradient)

	s = np.multiply(s, factor)
	s = cv2.convertScaleAbs(s)

	v = adaptiveGammaCorrection(v)
	v = adaptiveCLAHE(v)
	s = adaptiveCLAHE(s)
	

	final_CLAHE = cv2.merge((h,s,v))

	#additional sharpening
	tmpimg = cv2.cvtColor(final_CLAHE, cv2.COLOR_HSV2BGR)
	shimg = Image.fromarray(cv2.cvtColor(tmpimg,cv2.COLOR_BGR2RGB))
	sharpener = ImageEnhance.Sharpness(shimg)
	sharpened = sharpener.enhance(2.0)

	return sharpened
  

Path and save path

In [15]:
path = 'dark_dataset_10/grimace/'
save_path = './result/AGC/'

Loop Adaptive Gamma Correction

In [16]:
for subdir in os.listdir(path):
	ori_dir = path + subdir + '/'
	print(ori_dir)
	print(subdir)
	count_test = 0
	for filename in os.listdir(ori_dir):
		count_test = count_test + 1
		if filename.endswith(".JPG") or filename.endswith(".jpg") or filename.endswith(".png") or filename.endswith(".PNG"): 
			if not os.path.exists(save_path + ori_dir):
				os.makedirs(save_path + ori_dir)
			enhImg = wrapper(ori_dir + filename)
			img = enhImg.save(save_path + ori_dir + filename, quality=94 ,optimize=True)
			print(filename)
		#print('test1')
	#print('test2')
	print(count_test)
	print('===end of subdir===')

dark_dataset_10/grimace/and/
and
and_exp.1.jpg
and_exp.10.jpg
and_exp.11.jpg
and_exp.12.jpg
and_exp.13.jpg
and_exp.14.jpg
and_exp.15.jpg
and_exp.16.jpg
and_exp.17.jpg
and_exp.18.jpg
and_exp.19.jpg
and_exp.2.jpg
and_exp.20.jpg
and_exp.3.jpg
and_exp.4.jpg
and_exp.5.jpg
and_exp.6.jpg
and_exp.7.jpg
and_exp.8.jpg
and_exp.9.jpg
20
===end of subdir===
dark_dataset_10/grimace/ant/
ant
ant_exp.1.jpg
ant_exp.10.jpg
ant_exp.11.jpg
ant_exp.12.jpg
ant_exp.13.jpg
ant_exp.14.jpg
ant_exp.15.jpg
ant_exp.16.jpg
ant_exp.17.jpg
ant_exp.18.jpg
ant_exp.19.jpg
ant_exp.2.jpg
ant_exp.20.jpg
ant_exp.3.jpg
ant_exp.4.jpg
ant_exp.5.jpg
ant_exp.6.jpg
ant_exp.7.jpg
ant_exp.8.jpg
ant_exp.9.jpg
20
===end of subdir===
dark_dataset_10/grimace/chr/
chr
chr_exp.1.jpg
chr_exp.10.jpg
chr_exp.11.jpg
chr_exp.12.jpg
chr_exp.13.jpg
chr_exp.14.jpg
chr_exp.15.jpg
chr_exp.16.jpg
chr_exp.17.jpg
chr_exp.18.jpg
chr_exp.19.jpg
chr_exp.2.jpg
chr_exp.20.jpg
chr_exp.3.jpg
chr_exp.4.jpg
chr_exp.5.jpg
chr_exp.6.jpg
chr_exp.7.jpg
chr_exp.8.